In [11]:
#Get necesary libraries
import PIL as pil
import numpy as np

In [95]:
#Print versions
print(pil.__version__)
print(np.__version__)

7.0.0
1.18.1


In [78]:
#test Kernesl data structure
kernel = np.array([[-10,-10,0],[-10,0,10],[0,10,10]])
#print(l[0:, :2])
print(kernel)

[[-10 -10   0]
 [-10   0  10]
 [  0  10  10]]


In [96]:
#Function to "flip" de Kernell for corss correlation
def flipKernel(k):
    #Get array shape
    filas = k.shape[0]
    columnas = k.shape[1]
    
    #Create resulting array
    r = np.zeros((filas, columnas))
    #print(r)
    
    #support vars
    i = j = 0
    n = m = 0
    
    for x in k:
        for val in x:
            #print(k[i,j])
            n = filas-1-i
            m = columnas-1-j
            #print("Val: %s fila: %s Col: %s" % (val, n, m))
            r[n,m] = val
            
            if (j < columnas-1):
                j = j+1
            else:
                i += 1
                j = 0
    return r

flipKernel(np.array([[-10,-10,0],[-10,0,10],[0,10,10]]))

array([[ 10.,  10.,   0.],
       [ 10.,   0., -10.],
       [  0., -10., -10.]])

In [99]:
H = np.array([[10,20,0,30],[20,0,10,20],[10,0,40,10],[20,40,0,30]])
print(H)

[[10 20  0 30]
 [20  0 10 20]
 [10  0 40 10]
 [20 40  0 30]]


In [230]:
#kernell
#w = np.array([[-10,-10,0,-10,0],[-10,0,10,-10,0],[0,10,10,-10,0],[-10,0,10,-10,0],[0,10,10,-10,0]])
w = np.array([[-10,-10,0],[-10,0,10],[0,10,10]])
kR = w.shape[0]//2
kC = w.shape[1]//2

#image
H = np.array([[10,20,0,30],[20,0,10,20],[10,0,40,10],[20,40,0,30]])

#new shape to ensure resulting image is the expected size
sR = np.zeros((H.shape[0],kR))
nH = np.hstack((sR,H,sR))
#print(H)
#print("----------------------------")

sC = np.zeros((kC,nH.shape[1]))
nH = np.vstack((sC,nH,sC))
#print(H)
print("\n-------------- Kernel --------------")
print(w)
print("\n---------------- H ----------------")
print(H)
print("\n----------------nH ----------------")
print(nH.astype(int))
print("\n------------Start point------------")
print(kR,kC)


-------------- Kernel --------------
[[-10 -10   0]
 [-10   0  10]
 [  0  10  10]]

---------------- H ----------------
[[10 20  0 30]
 [20  0 10 20]
 [10  0 40 10]
 [20 40  0 30]]

----------------nH ----------------
[[ 0  0  0  0  0  0]
 [ 0 10 20  0 30  0]
 [ 0 20  0 10 20  0]
 [ 0 10  0 40 10  0]
 [ 0 20 40  0 30  0]
 [ 0  0  0  0  0  0]]

------------Start point------------
1 1


In [266]:
print("\n------------First value------------")

def applyKernel(w, H, pIni):
    x = pIni[0]
    y = pIni[1]
    
    r = 0
    
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            #print("---%s,%s" % (i-kR,j-kC))
            #print("Kernel: %s,%s" % (i,j))
            #print("Imagen %s,%s" % (x + (i-kR),y + (j-kC)))
            
            #print("---  %s, * %s" % (w[i,j],H[(x + (i-kR),y + (j-kC))]))
            
            r = r + (w[i,j] * H[(x + (i-kR),y + (j-kC))])
            #print(r)         
       
    return r


applyKernel(w, nH, (1,1))


------------First value------------


400.0

In [353]:
crossCorrH = np.zeros((H.shape))

for i in range(H.shape[0]):
    for j in range(H.shape[1]):
        #print(applyKernel(w, nH, (i+kR,j+kC)))
        crossCorrH[i,j] = applyKernel(w, nH, (i+kR,j+kC))
print("\n------------Cross corr------------\n")
print (crossCorrH)


------------Cross corr------------

[[ 400.    0.  400.  200.]
 [   0.    0.  500. -300.]
 [ 400.  500.  300. -400.]
 [ 300. -300. -500. -500.]]


In [354]:
convolH = np.zeros((H.shape))

for i in range(H.shape[0]):
    for j in range(H.shape[1]):
        #print(applyKernel(w, nH, (i+kR,j+kC)))
        convolH[i,j] = applyKernel(flipKernel(w), nH, (i+kR,j+kC))
        
print("\n------------Convolucion------------\n")
print (convolH)


------------Convolucion------------

[[-400.    0. -400. -200.]
 [   0.    0. -500.  300.]
 [-400. -500. -300.  400.]
 [-300.  300.  500.  500.]]


In [342]:
def normaliza(val, mn, mx, scaleMax=255):
    #    (0 + 500) *   255    / (1000)  
    r = (val + mx) * scaleMax / (mx - mn)
    #GENERALIZE (works but don't know why :D )
    return r

normaliza(0, rH.min(), rH.max())


127.5

In [359]:
def normH(H):
    normH = np.zeros((H.shape))
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            #print(normaliza(H[i,j], H.min(), H.max()))
            normH[i,j] = normaliza(H[i,j], H.min(), H.max())
            
    return normH

print("----------Normalizadas------------")
print("------------ x Corr --------------")
print(normH(crossCorrH))
print("------------ Convol --------------")
print(normH(convolH))

----------Normalizadas------------
------------ x Corr --------------
[[229.5 127.5 229.5 178.5]
 [127.5 127.5 255.   51. ]
 [229.5 255.  204.   25.5]
 [204.   51.    0.    0. ]]
------------ Convol --------------
[[ 25.5 127.5  25.5  76.5]
 [127.5 127.5   0.  204. ]
 [ 25.5   0.   51.  229.5]
 [ 51.  204.  255.  255. ]]
